In [1]:
from pyspark.sql import SparkSession
from delta import *

builder = (
    SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/nil/anaconda3/envs/spark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nil/.ivy2/cache
The jars for the packages stored in: /home/nil/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bb800a95-6fb4-4bb6-9b0d-ebb46cfdcb95;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 272ms :: artifacts dl 14ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

In [2]:
spark.sql(
    """CREATE OR REPLACE TABLE scd2Demo(
    pk1 INT,
    pk2 STRING,
    dim1 INT,
    dim3 int,
    dim4 int,
    active_status string,
    start_date timestamp,
    end_date timestamp
)
USING DELTA
LOCATION '/mnt/sda3/iNeuron/Data Engineering/pySpark/Delta_Lake/delta_lake_05'"""
)

24/02/17 14:40:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[]

In [4]:
spark.sql("select * from scd2Demo").show()

+---+---+----+----+----+-------------+----------+--------+
|pk1|pk2|dim1|dim3|dim4|active_status|start_date|end_date|
+---+---+----+----+----+-------------+----------+--------+
+---+---+----+----+----+-------------+----------+--------+



#### Time Travel

- Table History

In [ ]:
spark.sql("DESCRIBE HOSTORY sdc2Demo")

### PySpark Approach

#### Method 1: PySpark - Timestamp + Table

In [ ]:
df = (
    spark.read.format("delta")
    .option("timestampAsOf", "2022-05-15T09:55:53.000+0000")
    .table("scd2Demo")
)
df.show(truncate=False)

#### Method 2: PySpark - Timestamp + Path

In [ ]:
df = (
    spark.read.format("delta")
    .option("timestampAsOf", "2022-05-15T09:55:53.000+0000")
    .load("Delta_Lake/delta_lake_05")
)
df.show(truncate=False)

#### Method 3: PySpark - Version + Table

In [ ]:
df = (
    spark.read.format("delta").option("versionAsOf", 2).load("Delta_Lake/delta_lake_05")
)
df.show(truncate=False)

#### Method 4: PySpark - Version + Path

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 2).load("scd2Demo")
df.show(truncate=False)

### SQL Approach

#### Method 5: SQL - Version + Table

In [ ]:
spark.sql("select * from scd2Demo VERSION AS OF 2")

#### Method 6: SQL - Version + Path

In [ ]:
spark.sql("SELECT * `Delta_Lake/delta_lake_05` FROM VERSION AS OF 2")

#### Method 7: SQL - Timestamp + Table

In [ ]:
spark.sql("select * from scd2Demo TIMESTAMP AS OF 2022-05-15T09:55:53.000+0000")

#### Method 8: SQL - Timestamp + Path

In [ ]:
spark.sql(
    "select * from DELTA.`Delta_Lake/delta_lake_05` TIMESTAMP AS OF 2022-05-15T09:55:53.000+0000"
)